<a href="https://colab.research.google.com/github/jrtorresb/Pyspark_FIFA/blob/main/Pyspark_FIFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FIFA CHALLENGE

In [ ]:
# Install Spark in Google Colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install findspark
!pip install "pyspark==2.4.7" 


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

print(os.environ)

     |████████████████████████████████| 217.9MB 66kB/s 
     |████████████████████████████████| 204kB 18.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.7-py2.py3-none-any.whl size=218279465 sha256=6663b6e11c46807ff82012cbf8bcd8fe8c2f8e0fe102a14c12880245ae80bfdc
  Stored in directory: /root/.cache/pip/wheels/34/1f/2e/1e7460f80acf26b08dbb8c53d7ff9e07146f2a68dd5c732be5
Successfully built pyspark
environ({'NO_GCE_CHECK': 'True', 'GCS_READ_CACHE_BLOCK_SIZE_MB': '16', 'CLOUDSDK_CONFIG': '/content/.config', 'CUDA_VERSION': '10.1.243', 'PATH': '/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin', 'HOME': '/root', 'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64', 'LANG': 'en_US.UTF-8', 'SHELL': '/bin/bash', 'LIBRARY_PATH': '/usr/local/cuda/lib64/stubs', 'CUDA_PKG_VERSION': '10-1=10.1.243-1', 'SHLVL': '0', 'GCE_METADATA_TIMEOUT': '0', 'NCCL_VERSION': '2.8.3', 'NVID

In [ ]:
# Pyspark packages
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Python packages
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.express as px

In [ ]:
sc= SparkContext.getOrCreate()
spark= SparkSession(sc)

In [ ]:
%%time
# Read the CSV file
df_raw= spark.read.csv('fifa21_male2.csv', header= True)

CPU times: user 2.23 ms, sys: 77 µs, total: 2.31 ms
Wall time: 190 ms


In [ ]:
# Print Schema
df_raw.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- OVA: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- BOV: string (nullable = true)
 |-- BP: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Player Photo: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Flag Photo: string (nullable = true)
 |-- POT: string (nullable = true)
 |-- Team & Contract: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- Growth: string (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loan Date End: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Release Clause: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- Attacking: string (nullable = true)
 |-- Crossing: string (nu

In [ ]:
df_raw.show()

+---+-------------+---+---+--------------------+------------------+---+---+------------+--------------------+--------------------+--------------------+---+--------------------+------+------+-----+------+------------+-------------+-----+----+--------------+------------+---------+--------+---------+----------------+-------------+-------+-----+---------+-----+-----------+------------+------------+--------+------------+------------+-------+---------+-------+-----+----------+-------+-------+--------+----------+---------+----------+-------------+-----------+------+---------+---------+---------+-------+---------------+--------------+-----------+---------+-----------+----------+--------------+-----------+-----------+----------+---+---+------+------+---+---+---+---+---+---+---+----+----+----+----+----+-----+-----+-----+----+-----+-----+-----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+------+
| ID|         Name|Age|OVA|         Nationality|         

In [ ]:
# Number of records
df_raw.count()

17125

In [ ]:
# Temporary View
df_raw.registerTempTable("fifa21")

In [ ]:
df_raw.groupBy('Nationality').count().show()

+--------------+-----+
|   Nationality|count|
+--------------+-----+
|          Chad|    2|
|      Paraguay|  180|
|        Russia|  114|
|Chinese Taipei|    3|
|       Senegal|  141|
|        Sweden|  260|
|        Guyana|    9|
|   Philippines|    3|
|       Eritrea|    1|
|      Malaysia|    2|
|     Singapore|    1|
|        Turkey|  226|
|        Malawi|    1|
|          Iraq|    4|
|       Germany| 1154|
|       Comoros|   10|
|   Afghanistan|    3|
|   Ivory Coast|  119|
|        Jordan|    2|
|        Rwanda|    1|
+--------------+-----+
only showing top 20 rows



*  We need write one folder for each Nationality, each folder should contains
the players with that Nationality

In [ ]:
nationality=spark.sql('SELECT distinct Nationality FROM fifa21').toPandas()

In [ ]:
nations=nationality.values.ravel().tolist()

In [ ]:
# Create data frames in a dictionary with Players with that Nationality
data_frames={}

for i in nations:
  data_frames[i] = df_raw.filter("Nationality == '"+ i + "'")
  data_frames[i].coalesce(1).write \
                      .format('orc') \
                      .mode('overwrite') \
                      .save('df_'+i+'_out_orc')


# If you want to save each data frame you can use   data_frames.toPandas().to_csv("name")

* We need to know who are the 10 top players for each position, you should
write this data too in another output folder

In [ ]:
position = spark.sql('SELECT distinct Position FROM fifa21').toPandas()
position = position.values.ravel().tolist()
position = [i for i in position if i]  # Remove None

data_frames_position={}
for i in position:
  data_frames_position[i] = df_raw.filter("Position == '"+ i + "'").orderBy('OVA', ascending=False).toPandas()

# Foer each data frame you can take de first ten rows in each dataframe and you can save to csv as to_csv("name")
# Save as orc file
for i in position:
  data_frames_position[i] = df_raw.filter("Position == '"+ i + "'").orderBy('OVA', ascending=False)
  data_frames_position[i].coalesce(1).write \
                      .format('orc') \
                      .mode('overwrite') \
                      .save('df_'+i+'out_orc')

* We need how many players for each position each Club.

In [ ]:
df_raw.groupBy('Club','Position').count().show()

+--------------------+------------+-----+
|                Club|    Position|count|
+--------------------+------------+-----+
|             FC Sion|      CDM CM|    1|
|          Al Ittihad|      CDM CM|    1|
|    Lokomotiv Moscow|          LM|    2|
|        Club América|   CAM ST CF|    1|
|         Aston Villa|          LB|    1|
|           Hibernian|          LB|    2|
|           FC Lugano|   ST LW CAM|    1|
|      BSC Young Boys|      CB CDM|    1|
|             Chelsea|          ST|    3|
|        Hamburger SV|          ST|    5|
|West Bromwich Albion|       ST RW|    1|
|           Blackpool|          ST|    3|
|           FC Juárez|   CAM RM CF|    1|
|                Roma|       RB LB|    1|
|          Ettifaq FC|      CAM LM|    1|
|        Boca Juniors|       RM RW|    1|
|           Sydney FC|CAM RM CM ST|    1|
|           AS Monaco|          ST|    3|
|          AZ Alkmaar|          GK|    5|
|      Club Brugge KV|      CAM CM|    1|
+--------------------+------------

* We need to know the top 10 clubs for sprint speed average.

In [113]:
sprint = df_raw.groupBy('Club','Sprint Speed').mean().orderBy('Sprint Speed', ascending=False)
sprint.show()

+--------------------+------------+
|                Club|Sprint Speed|
+--------------------+------------+
| Paris Saint-Germain|          96|
|Wolverhampton Wan...|          96|
|           Dijon FCO|          96|
|   FC Bayern München|          96|
|               Inter|          95|
|   Manchester United|          95|
|      Club Brugge KV|          95|
|    Ulsan Hyundai FC|          95|
|         Real Madrid|          95|
| Bayer 04 Leverkusen|          95|
|   FC Bayern München|          95|
|     Rosario Central|          94|
|             Watford|          94|
|           Junior FC|          94|
|            Portugal|          94|
|            Cracovia|          94|
|      BB Erzurumspor|          94|
|Athletic Club de ...|          94|
|     Adelaide United|          94|
|   Vitória Guimarães|          94|
+--------------------+------------+
only showing top 20 rows



In [114]:
#Save as orc file
sprint.coalesce(1).write \
                    .format('orc') \
                    .mode('overwrite') \
                    .save('dsprint_out_orc')

In [ ]:
df_raw.show()

+---+-------------+---+---+--------------------+------------------+---+---+------------+--------------------+--------------------+--------------------+---+--------------------+------+------+-----+------+------------+-------------+-----+----+--------------+------------+---------+--------+---------+----------------+-------------+-------+-----+---------+-----+-----------+------------+------------+--------+------------+------------+-------+---------+-------+-----+----------+-------+-------+--------+----------+---------+----------+-------------+-----------+------+---------+---------+---------+-------+---------------+--------------+-----------+---------+-----------+----------+--------------+-----------+-----------+----------+---+---+------+------+---+---+---+---+---+---+---+----+----+----+----+----+-----+-----+-----+----+-----+-----+-----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+------+
| ID|         Name|Age|OVA|         Nationality|         

*  Calculate the IMC for each player, we need to know all players with
overweight (IMC>25).

In [ ]:
df_raw = df_raw.withColumn('Height', regexp_replace('Height',"'", ""))
df_raw = df_raw.withColumn('Height', regexp_replace('Height','""', ""))
df_raw = df_raw.withColumn('Weight', regexp_replace('Weight', "lbs", ""))
df_raw.show()

+---+-------------+---+---+--------------------+------------------+---+---+------------+--------------------+--------------------+--------------------+---+--------------------+------+------+-----+------+------------+-------------+-----+----+--------------+------------+---------+--------+---------+----------------+-------------+-------+-----+---------+-----+-----------+------------+------------+--------+------------+------------+-------+---------+-------+-----+----------+-------+-------+--------+----------+---------+----------+-------------+-----------+------+---------+---------+---------+-------+---------------+--------------+-----------+---------+-----------+----------+--------------+-----------+-----------+----------+---+---+------+------+---+---+---+---+---+---+---+----+----+----+----+----+-----+-----+-----+----+-----+-----+-----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+------+
| ID|         Name|Age|OVA|         Nationality|         

In [ ]:
# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 
# List of continuous features
CONTI_FEATURES  = ['Height','Weight']
# Convert the type
df_numeric_height_weight = convertColumn(df_raw, CONTI_FEATURES, FloatType())
# Check the dataset
df_numeric_height_weight.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- OVA: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- BOV: string (nullable = true)
 |-- BP: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Player Photo: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Flag Photo: string (nullable = true)
 |-- POT: string (nullable = true)
 |-- Team & Contract: string (nullable = true)
 |-- Height: float (nullable = true)
 |-- Weight: float (nullable = true)
 |-- foot: string (nullable = true)
 |-- Growth: string (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loan Date End: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Release Clause: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- Attacking: string (nullable = true)
 |-- Crossing: string (null

In [ ]:
df_numeric_height_weight.show()

+---+-------------+---+---+--------------------+------------------+---+---+------------+--------------------+--------------------+--------------------+---+--------------------+------+------+-----+------+------------+-------------+-----+----+--------------+------------+---------+--------+---------+----------------+-------------+-------+-----+---------+-----+-----------+------------+------------+--------+------------+------------+-------+---------+-------+-----+----------+-------+-------+--------+----------+---------+----------+-------------+-----------+------+---------+---------+---------+-------+---------------+--------------+-----------+---------+-----------+----------+--------------+-----------+-----------+----------+---+---+------+------+---+---+---+---+---+---+---+----+----+----+----+----+-----+-----+-----+----+-----+-----+-----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+------+
| ID|         Name|Age|OVA|         Nationality|         

In [ ]:
df_numeric_height_weight = df_numeric_height_weight.withColumn('IMC', df_numeric_height_weight.Weight/df_numeric_height_weight.Height**2 * 703)
df_numeric_height_weight.show()

+---+-------------+---+---+--------------------+------------------+---+---+------------+--------------------+--------------------+--------------------+---+--------------------+------+------+-----+------+------------+-------------+-----+----+--------------+------------+---------+--------+---------+----------------+-------------+-------+-----+---------+-----+-----------+------------+------------+--------+------------+------------+-------+---------+-------+-----+----------+-------+-------+--------+----------+---------+----------+-------------+-----------+------+---------+---------+---------+-------+---------------+--------------+-----------+---------+-----------+----------+--------------+-----------+-----------+----------+---+---+------+------+---+---+---+---+---+---+---+----+----+----+----+----+-----+-----+-----+----+-----+-----+-----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+------+-------------------+
| ID|         Name|Age|OVA|         N

In [ ]:
df_numeric_height_weight_filter = df_numeric_height_weight.filter('IMC > 25')
df_numeric_height_weight_filter.show()

+----+--------------+---+---+-------------------+--------------------+---+---+------------+--------------------+--------------------+--------------------+---+--------------------+------+------+-----+------+------------+-------------+-----+----+--------------+------------+---------+--------+---------+----------------+-------------+-------+-----+---------+-----+-----------+------------+------------+--------+------------+------------+-------+---------+-------+-----+----------+-------+-------+--------+----------+---------+----------+-------------+-----------+------+---------+---------+---------+-------+---------------+--------------+-----------+---------+-----------+----------+--------------+-----------+-----------+----------+---+---+------+------+---+---+---+---+---+---+---+----+----+----+----+----+-----+-----+-----+----+-----+-----+-----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+----+----+----+-----+----+------+------------------+
|  ID|          Name|Age|OVA|        N

In [ ]:
#Save as orc file
df_numeric_height_weight_filter.coalesce(1).write \
                    .format('orc') \
                    .mode('overwrite') \
                    .save('df_numeric_height_weight_filter_out_orc')